In [17]:
#Enables module automatic reload.
#Your notebook will be able to pick up code updates made to qiskit-metal (or other) module code.

%reload_ext autoreload
%autoreload 2

In [18]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

design = designs.DesignPlanar()
gui = MetalGUI(design)

# If you disable the next line with "overwrite_enabled", then you will need to
# delete a component [<component>.delete()] before recreating it.
design.overwrite_enabled = True

In [19]:
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

#Explore the options of the LaunchpadWirebondCoupled.
LaunchpadWirebondCoupled.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'trace_width': 'cpw_width',
 'trace_gap': 'cpw_gap',
 'coupler_length': '62.5um',
 'lead_length': '25um'}

In [4]:
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

#Explore the options of the RouteMeander.
RouteMeander.get_template_options(design)

{'chip': 'main',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False}

In [5]:
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround

#Explore the options of the OpenToGround.
OpenToGround.get_template_options(design)
# ShortToGround.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'width': '10um'}

In [20]:
#Setup the launchpad location and orientation.
launch_options = dict(pos_x='990um', pos_y='2812um', orientation='270', lead_length='30um')

lp = LaunchpadWirebondCoupled(design, 'P4_Q', options = launch_options)

In [21]:
#Setup the OpenToGround location and orientation.
otg_options = dict(pos_x='2.5mm',  pos_y='0.5mm', orientation='0')

otg = OpenToGround(design, 'open_to_ground', options=otg_options)
# otg = ShortToGround(design, 'short_to_ground', options=otg_options)

In [22]:
#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options = Dict(
        total_length='10 mm',
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component=lp.name, pin='tie'),
            end_pin=Dict(component=otg.name, pin='open')), )

meander = RouteMeander(design, 'bus',  options=meander_options)
gui.rebuild()
gui.autoscale()